## Orders Truck Optimized Algorithm:

### Step 1 : Library Imports 

In [45]:
import csv
import math
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import sys

### Step 2 : Constants Declaration

In [46]:
MAX_DELIVERIES = 10
DRIVE_SPEED_KMPH = 20
PRICE_PER_TRUCK = 100
PRICE_PER_KM = 0.061
MAX_TIME_PER_TRUCK = 10
TOTAL_CLUSTERS = 51

In [47]:
### Step 4 : Functions Defition

In [48]:
def distance(x1, y1, x2, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

### Step 5 : Project Models

#### Model 1 : Truck 

In [49]:
class Truck:
    id = 0

    def __init__(self):
        self.capacity_left = MAX_DELIVERIES
        self.x = 0
        self.y = 0
        self.current_time = 0
        self.id = Truck.id
        self.deliveries = []  # [truck_id, order_id, seq_no]
        self.current_order = 1
        self.delivered = False
        Truck.id += 1

    def can_deliver_order(self, order):
        """
        Checks whether the truck can deliver the order and reach the depot in
        time.
        """
        if self.capacity_left == 0:
            return False

        time_to_reach_order = distance(
            self.x, self.y, order.x, order.y
        ) / DRIVE_SPEED_KMPH

        if self.current_time + time_to_reach_order >= order.end_time:
            return False

        time_to_reach_depot = distance(
            order.x, order.y, 0, 0
        ) / DRIVE_SPEED_KMPH

        if (
            max(self.current_time + time_to_reach_order, order.start_time)
                + time_to_reach_depot > MAX_TIME_PER_TRUCK
        ):
            return False

        print("True")
        return True

    def deliver_order(self, order):
        dist = distance(self.x, self.y, order.x, order.y)

        # update current_time
        self.current_time = max(
            self.current_time + (dist / DRIVE_SPEED_KMPH),
            order.start_time
        )

        # update capacity
        self.capacity_left -= 1

        # update location
        self.x = order.x
        self.y = order.y

        # add order to deliveries
        self.deliveries.append([self.id, order.id, self.current_order])
        self.current_order += 1

    def deliver(self):
        self.delivered = True
        return self.deliveries[::]

    def has_delivered(self):
        return self.delivered

#### Model 2 : Order

In [50]:
class Order:
    def __init__(self, id, x, y, start_time):
        self.id = id
        self.x = x
        self.y = y
        self.start_time = start_time
        self.end_time = self.start_time + 1

### Step 5 :Routing Optimization Algorithm :

In [51]:
def clustered(orders, kmeans):
    clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}
    deliveries = []

    current_truck = Truck(orders[::])

    # loop over each cluster
    for cluster in clusters.values():
        curr_len = len(deliveries)

        orders_in_cluster = [orders[idx] for idx in cluster]

        for order in sorted(orders_in_cluster, key=lambda x: x.end_time):
            if current_truck.can_deliver_order(order):
                current_truck.deliver_order(order)
            else:
                deliveries.extend(current_truck.deliver())
                current_truck = Truck(orders[::])
                current_truck.deliver_order(order)

    if not current_truck.has_delivered():
        deliveries.extend(current_truck.deliver())

    return deliveries

### Step 6 : Finding clusters, running algo, formatting output :

In [52]:
def answer():
    df = pd.read_csv("orders.csv")

    orders_to_cluster = []
    orders = []

    for _, row in df.iterrows():
        orders_to_cluster.append([row["x"], row["y"]])
        order = Order(
            row["order_id"], row["x"], row["y"], row["time_window_start"]
        )
        orders.append(order)

    kmeans = KMeans(n_clusters=TOTAL_CLUSTERS, random_state=0)
    kmeans.fit(orders_to_cluster)

    # ans_list = bruteforce(orders)
    ans_list = clustered(orders, kmeans)

    print(len(ans_list))

    with open("answer.txt", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["truck_id", "order_id", "sequence_number"])
        for row in ans_list:
            writer.writerow(row)


### Result Evaluation :

In [53]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20,40)
import matplotlib.pyplot as plt
%matplotlib inline  

"""
Can be used to validate and score your solution
Usage: validate.py <file name>
"""

import sys
import math
import pandas as pd


MAX_DELIVERIES = 10
DRIVE_SPEED_KMPH = 20
PRICE_PER_TRUCK = 100
PRICE_PER_KM = 0.061

def distance1(pos1, pos2):
    """Find the euclidean distance between two, two dimentional tuples"""
    diff_1 = pos1[0] - pos2[0]
    diff_2 = pos1[1] - pos2[1]
    return math.sqrt(diff_1*diff_1 + diff_2*diff_2)


def is_valid(data_frame):
    """Check if the route provided for a truck is legal.
       Print any problems to stderr"""
    epsilon = 0.00001
    check = data_frame.copy()
    check.sort_values("sequence_number", inplace=True)
    location = (0, 0)
    time = 0
    valid = []
    # Check all deliveries make their time windows
    row = ""
    for _, row in check.iterrows():
        new_location = (row.x, row.y)
        travel_time_hours = distance1(location, new_location) / DRIVE_SPEED_KMPH
        time = time + travel_time_hours
        if time > row.time_window_start+1:
            sys.stderr.write("Error: Truck " + str(row.truck_id)
                             + " delivers to " + str(row.order_id)
                             + " late at " + str(time) + "\n")
            valid.append(False)
        else:
            valid.append(True)
        if time - epsilon < row.time_window_start:
            time = row.time_window_start
        location = new_location

    # Check the van returns to the depot in time
    new_location = (0, 0)
    travel_time_hours = distance1(location, new_location) / DRIVE_SPEED_KMPH
    time = time + travel_time_hours
    if time - epsilon > 10:
        sys.stderr.write("Error: Truck " + str(row.truck_id)
                         + " returns to the depot at " + str(time) + "\n")
        valid = [False] * len(check)

    # Check the van makes at most 10 deliveries
    if len(check) > MAX_DELIVERIES:
        sys.stderr.write("Error: Truck " + str(row.truck_id)
                         + " delivers to " + str(len(check)) + " orders.\n")
        valid = [False] * len(check)
    return valid
    
def route_distance(data_frame):
    """Check if the route provided for a truck is legal.
       Print any problems to stderr"""
    check = data_frame.copy()
    check.sort_values("sequence_number", inplace=True)
    location = (0, 0)
    total_distance = 0
    # Sum the distances to last order
    for _, row in check.iterrows():
        new_location = (row.x, row.y)
        total_distance += distance1(location, new_location)
        location = new_location

    # Add distance to depot
    new_location = (0, 0)
    total_distance += distance1(location, new_location)

    return total_distance

def check(file_name):
    """
    Find the score for the solution and report any problems
    """
    # Columns are: truck_id, order_id, sequence_number
    data = pd.read_csv(file_name)
    data.columns = [column.strip() for column in data.columns.tolist()]
    # Columns are: order_id, x, y, time_window_start
    orders = pd.read_csv("orders.csv")
    orders.columns = [column.strip() for column in orders.columns.tolist()]
    data = data.merge(orders, on="order_id")

    # Check correct unique orders
    specified_orders = data.order_id.nunique()
    if specified_orders != len(orders):
        sys.stderr.write("Error: " + str(specified_orders) + " specified. "
                         + str(len(orders)) + " orders required.\n")
        exit(1)

    # Check specified duplicate orders
    duplicates = data.order_id.duplicated()
    if sum(duplicates):
        msg = "Error: the following order(s) have multiple deliveries:\n"
        duplicated_orders = data[duplicates].groupby("order_id")
        duplicated_orders = duplicated_orders.mean().reset_index().order_id
        duplicated_orders = duplicated_orders.values.tolist()
        sys.stderr.write(msg + str(duplicated_orders) + "\n")
        exit(1)

    # Check the validity of the routes
    valid = data.groupby("truck_id").apply(is_valid).tolist()
    valid = [x for sublist in valid for x in sublist]
    total_distance = sum(data.groupby("truck_id").apply(route_distance))

    if not sum(valid) == len(data):
        print("Invalid solution")
        exit(1)
    else:
        vans = data.truck_id.nunique()
        #print("Correct solution found with "
             # + str(vans) + " trucks.")
        #print("Total distance travelled: {0:.2f}".format(total_distance)
             # + " km")
        cost = vans*100 + total_distance*0.061
        #print("Total cost: £{0:.2f}".format(cost))
        return (vans,total_distance,cost)
        


In [54]:
def pipeline():
    answer()
    results = check("answer.txt")
    return results
    
results = pipeline()
results

TypeError: __init__() takes 1 positional argument but 2 were given